# Semantic Kernel mit OpenBnB MCP Server-Integration

Dieses Notebook zeigt, wie man Semantic Kernel mit dem tatsächlichen OpenBnB MCP-Server verwendet, um mithilfe des MCPStdioPlugin nach echten Airbnb-Unterkünften zu suchen. Für den Zugriff auf LLM wird Azure AI Foundry verwendet. Um Ihre Umgebungsvariablen einzurichten, können Sie der [Setup-Lektion](/00-course-setup/README.md) folgen.


## Importieren der benötigten Pakete


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Erstellen der MCP-Plugin-Verbindung

Wir verbinden uns mit dem [OpenBnB MCP-Server](https://github.com/openbnb-org/mcp-server-airbnb) mithilfe von MCPStdioPlugin. Dieser Server bietet Suchfunktionen für Airbnb über das @openbnb/mcp-server-airbnb-Paket.


## Erstellen des Clients

In diesem Beispiel verwenden wir Azure AI Foundry für den Zugriff auf unser LLM. Stellen Sie sicher, dass Ihre Umgebungsvariablen korrekt eingerichtet sind.


## Umgebungskonfiguration

Konfigurieren Sie die Azure OpenAI-Einstellungen. Stellen Sie sicher, dass die folgenden Umgebungsvariablen gesetzt sind:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Verständnis der OpenBnB MCP-Integration

Dieses Notebook verbindet sich mit dem **echten OpenBnB MCP-Server**, der die tatsächliche Airbnb-Suchfunktionalität bereitstellt.

### So funktioniert es:

1. **MCPStdioPlugin**: Nutzt die Kommunikation über Standard-Ein-/Ausgabe mit dem MCP-Server
2. **Echtes NPM-Paket**: Lädt und führt `@openbnb/mcp-server-airbnb` über npx aus
3. **Live-Daten**: Liefert echte Airbnb-Immobiliendaten aus deren APIs
4. **Funktionsentdeckung**: Der Agent erkennt automatisch die verfügbaren Funktionen des MCP-Servers

### Verfügbare Funktionen:

Der OpenBnB MCP-Server stellt typischerweise folgende Funktionen bereit:
- **search_listings** - Suche nach Airbnb-Immobilien basierend auf Standort und Kriterien
- **get_listing_details** - Abrufen detaillierter Informationen zu bestimmten Immobilien
- **check_availability** - Verfügbarkeit für bestimmte Daten prüfen
- **get_reviews** - Bewertungen für Immobilien abrufen
- **get_host_info** - Informationen über Gastgeber von Immobilien abrufen

### Voraussetzungen:

- **Node.js** auf Ihrem System installiert
- **Internetverbindung**, um das MCP-Server-Paket herunterzuladen
- **NPX** verfügbar (kommt mit Node.js)

### Verbindung testen:

Sie können den MCP-Server manuell testen, indem Sie folgendes ausführen:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dies lädt und startet den OpenBnB MCP-Server, mit dem Semantic Kernel dann echte Airbnb-Daten abruft.


## Ausführen des Agenten mit dem OpenBnB MCP-Server

Jetzt werden wir den KI-Agenten ausführen, der sich mit dem OpenBnB MCP-Server verbindet, um echte Airbnb-Unterkünfte in Stockholm für 2 Erwachsene und 1 Kind zu suchen. Sie können die `user_inputs`-Liste gerne ändern, um die Suchkriterien anzupassen.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Zusammenfassung
Herzlichen Glückwunsch! Sie haben erfolgreich einen KI-Agenten entwickelt, der mit der realen Unterkunftssuche über das Model Context Protocol (MCP) integriert ist:

## Verwendete Technologien:
- Semantic Kernel - Zum Aufbau intelligenter Agenten mit Azure OpenAI
- Azure AI Foundry - Für LLM-Funktionen und Chat-Komplettierung
- MCP (Model Context Protocol) - Für standardisierte Tool-Integration
- OpenBnB MCP Server - Für echte Airbnb-Suchfunktionen
- Node.js/NPX - Zum Ausführen des externen MCP-Servers

## Was Sie gelernt haben:
- MCP-Integration: Verbindung von Semantic Kernel-Agenten mit externen MCP-Servern
- Echtzeit-Datenzugriff: Suche nach tatsächlichen Airbnb-Unterkünften über Live-APIs
- Protokollkommunikation: Nutzung von Stdio-Kommunikation zwischen Agent und MCP-Server
- Funktionsentdeckung: Automatische Erkennung verfügbarer Funktionen von MCP-Servern
- Streaming-Antworten: Erfassen und Protokollieren von Funktionsaufrufen in Echtzeit
- HTML-Rendering: Formatierung von Agentenantworten mit gestalteten Tabellen und interaktiven Anzeigen

## Nächste Schritte:
- Integration zusätzlicher MCP-Server (Wetter, Flüge, Restaurants)
- Aufbau eines Multi-Agenten-Systems, das MCP- und A2A-Protokolle kombiniert
- Erstellung eigener MCP-Server für Ihre eigenen Datenquellen
- Implementierung von persistentem Gesprächsspeicher über Sitzungen hinweg
- Bereitstellung des Agenten in Azure Functions mit MCP-Server-Orchestrierung
- Hinzufügen von Benutzerauthentifizierung und Buchungsfunktionen



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
